Importing libraries and cleaning up the data set:

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path  


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm


dem_candidates = pd.read_csv('../datasets/dem_candidates.csv')

### Sara -- Frequentist Model using statsmodels.api
Why: From lab 5:
Let's start by considering the problem from a frequentist lens. To do this, we'll use the statsmodels.api, which allows us to create a model in just a few lines of code.

After fitting our model, we can call the .summary() method, and get a breakdown of our model and some details on how well it fit our data.



2a) Fit Poisson GLM model where Temp_Anomaly is a covariate (exogenous variable): No need to modify
freq_model = sm.GLM(df["Num_Storms"], exog = sm.add_constant(df["Temp_Anomaly"]), 
                  family=sm.families.Poisson())
freq_res = freq_model.fit()
print(freq_res.summary())

In [129]:
predictors = [ 'Partisan Lean', 'Race', 'Veteran?', 'LGBTQ?', 'Self-Funder?', 'STEM?','Obama Alum?', 'Elected Official?','total_runners']
# Fit Poisson GLM model where Temp_Anomaly is a covariate (exogenous variable): No need to modify
freq_model = sm.GLM(endog = house["General Status"], exog = sm.add_constant(house[predictors]), 
                  family=sm.families.Binomial())
freq_res = freq_model.fit()
print(freq_res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:         General Status   No. Observations:                  633
Model:                            GLM   Df Residuals:                      623
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -335.48
Date:                Wed, 06 Dec 2023   Deviance:                       670.95
Time:                        23:49:03   Pearson chi2:                     622.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.09850
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.2869      0.33

using predictors that have no relationship with the response tends to cause a deterioration in the test error rate (since such predictors cause an increase in variance without a corresponding decrease in bias), and so removing such predictors may in turn yield an improvement.

In [130]:
# train = (Smarket.Year < 2005)
# Smarket_train = Smarket.loc[train]
# Smarket_test = Smarket.loc[∼train]

# X_train, X_test = X.loc[train], X.loc[∼train]
# y_train, y_test = y.loc[train], y.loc[∼train]
# glm_train = sm.GLM(
#     y_train,
#     X_train,
#     family=sm.families.Binomial()
# )
# results = glm_train.fit()
# probs = results.predict(exog=X_test)

## Fitting a Frequentist Logistic Regression model using Sklearn:
To choose the predictors (X) we used domain knowledge. 
- X: Predictors of General Status
- Y: General Status (1 if the candidate advanced to the general election, 0 if not)

In [131]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


#setting up the logistic regression model by splitting into train and test
train, test = train_test_split(house, test_size = .30, random_state = 101)
predictors = [ 'Partisan Lean', 'Race', 'Veteran?', 'LGBTQ?', 'Self-Funder?', 'STEM?','Obama Alum?', 'total_runners']
X_train, y_train = train[predictors], train['General Status']
X_test, y_test = test[predictors], test['General Status']

In [132]:
# X_train = train[['Partisan Lean', 'Race',
#        'Veteran?', 'LGBTQ?', 'Elected Official?', 'Self-Funder?', 'STEM?',
#        'Obama Alum?', 'Party Support?', 'Emily Endorsed?', 'total_runners']]
# y_train = train['General Status']

# X_test = test[['Partisan Lean', 'Race',
#        'Veteran?', 'LGBTQ?', 'Elected Official?', 'Self-Funder?', 'STEM?',
#        'Obama Alum?', 'Party Support?', 'Emily Endorsed?', 'total_runners']]
# y_test = test['General Status']

logisticmodel = LogisticRegression(penalty='none', solver='lbfgs')

logisticmodel.fit(X_test, y_test)

probs = logisticmodel.predict_proba(X_test)[:, 1]
y_hat = (probs > 0.5).astype(int)

accuracy = np.mean(y_test == y_hat)
print(f"Accuracy on test set: {accuracy}")

Accuracy on test set: 0.7631578947368421


We first created a Logistic Regression model using a subset of features in the house dataset. We employed feature engineering to include a column of how many total runners were participating in the election. Because the data on each candidate is not independent since the outcome of a race for a single candidate is affected by the outcome for another candidate in that same race, we included the number of total runners. Additionally, because both Biden and Warren only endorsed 5 candidates each, we excluded those features. Emily's list endorsed 42 candidates in the 2018 house of representative elections, so we intentionally included this column. There were 33 Obama Alums, so we included this as well. 

In [133]:
house['Obama Alum?'].sum()

26.0

## Fitting a Logistic Regression model using statsmodel:

In [134]:
predictors = [ 'Partisan Lean', 'Race', 'Veteran?', 'LGBTQ?', 'Self-Funder?', 'STEM?','Obama Alum?', 'Elected Official?','total_runners']
X = house[predictors]  # independent variables

# Add a constant term to the independent variables
X = sm.add_constant(X)

y = house['General Status']

# Fit the logistic regression model
model = sm.Logit(y, X)
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.529978
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:         General Status   No. Observations:                  633
Model:                          Logit   Df Residuals:                      623
Method:                           MLE   Df Model:                            9
Date:                Wed, 06 Dec 2023   Pseudo R-squ.:                 0.08911
Time:                        23:49:03   Log-Likelihood:                -335.48
converged:                       True   LL-Null:                       -368.29
Covariance Type:            nonrobust   LLR p-value:                 1.084e-10
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.2869      0.334      0.859      0.390      -0.368       0.941
Partisan

In [135]:
print(result.aic, result.bic)

690.9515452801878 735.4562495016296


While our initial model had a decent accuracy of approximately 84%, we can further optimize our model by selecting the best combination of features. As we can see in the summary from the model above, the log-likelihood is pretty low. Additionally, when we interpret the confidence intervals for the probabilities of the coefficients, we can see that many of these intervals include 0. Thus, it would make sense to re-evaluate these features or take them out. Additionally, when comparing the coefficients, we can see that some of the coefficients only have a marginal affect on the dependent variable, such as 'LGBTQ?' or 'Self-Funder?'.

Forward selection of features:

In [136]:
predictors = [ 'Partisan Lean', 'Race', 'Veteran?', 'LGBTQ?', 'Self-Funder?', 'STEM?','Obama Alum?', 'Elected Official?','total_runners']
all_predictors_df = house[predictors]


y = house['General Status']

best_features = []
best_aic = float('inf') 

for feature in all_predictors_df.columns:
    # Add a constant term and the current feature
    X = sm.add_constant(all_predictors_df[best_features + [feature]])

    # Fit the logistic regression model
    model = sm.Logit(y, X)
    result = model.fit()

    # Check AIC and update if it is lower
    if result.aic < best_aic:
        best_aic = result.aic
        best_features.append(feature)

# Fit the final model with the best features
X_final = sm.add_constant(all_predictors_df[best_features])
final_model = sm.Logit(y, X_final)
final_result = final_model.fit()


print(final_result.summary(), final_result.aic)

Optimization terminated successfully.
         Current function value: 0.574478
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.574083
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.574473
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.574478
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.574388
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.570005
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.566736
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.558117
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531759
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.531759
  

@ Nikki I am not sure what this is for? 
train, test = train_test_split(all_predictors_df, test_size = .30, random_state = 101)

Backward selection of features: 

In [137]:
predictors = [ 'Partisan Lean', 'Race', 'Veteran?', 'LGBTQ?', 'Self-Funder?', 'STEM?','Obama Alum?', 'Elected Official?','total_runners']
all_predictors_df = house[predictors]
y = house['General Status']

X = sm.add_constant(all_predictors_df)  #.drop returns a df
X_shuffled = X.sample(frac=1, axis=1, random_state=42)


# Fit the initial model with all features
initial_model = sm.Logit(y, X_shuffled)
initial_result = initial_model.fit()
best_aic = initial_result.aic
best_bic = initial_result.bic
best_model = initial_model
selected_features = list(X_shuffled.columns)

# Backward selection

# @ Nikki why do we drop the constant
for feature in X_shuffled.columns[0:]:  # Exclude the constant term
    # Fit the model without the current feature
    X_subset = X_shuffled[selected_features].drop(feature, axis=1)
    model = sm.Logit(y, X_subset)
    result = model.fit()

    # Compare AIC and BIC
    if result.aic < best_aic and result.bic < best_bic:
        best_aic = result.aic
        best_bic = result.bic
        best_model = model
        selected_features.remove(feature)

# Fit the final model with the selected features
final_result = best_model.fit()

# Print the summary of the final model
print(final_result.summary())

Optimization terminated successfully.
         Current function value: 0.529978
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.539299
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.531212
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.532558
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.535826
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.535916
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.532589
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.571044
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.532723
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.532730
  

Testing the best recommended features in a logistic regression model with sklearn: 
We will only take the four most signifcant regressors according to the forward and backwrard seelction modles/


In [138]:
#house_important_only = house[house['Elected Official','STEM?','Obama Alum?', 'total_runners', 'General Status']]
train, test = train_test_split(house, test_size = .30, random_state = 101)
new_predictors = ['Elected Official?','STEM?','Obama Alum?', 'total_runners']
X_train1, y_train1 = train[new_predictors], train['General Status']
X_test1, y_test1 = test[new_predictors], test['General Status']

logisticmodel1 = LogisticRegression(
    penalty='none', solver='lbfgs'
)

logisticmodel1.fit(X_test1, y_test1)

probs = logisticmodel1.predict_proba(X_test1)[:, 1]
y_hat = (probs > 0.5).astype(np.int64)

accuracy = np.mean(y_test == y_hat)
print(f"Accuracy on test set: {accuracy}")

Accuracy on test set: 0.7368421052631579


Nonparametric models: Random Forest

In [139]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#Using the optimal features selected in the previous section: -- dominic said to do all possible ones. 

X_cols = [ 'Partisan Lean', 'Race', 'Veteran?', 'LGBTQ?', 'Self-Funder?', 'STEM?','Obama Alum?', 'Elected Official?','total_runners']
y_col = 'General Status'
forest_model = RandomForestClassifier()

train, test = train_test_split(house, test_size = .25, random_state = 101)
forest_model.fit(train[X_cols], train['General Status'])

train["forest_pred"] = forest_model.predict(train[X_cols])
test["forest_pred"] = forest_model.predict(test[X_cols])

In [140]:
train_rmse = np.mean((train["forest_pred"] - train[y_col]) ** 2) ** 0.5
test_rmse = np.mean((test["forest_pred"] - test[y_col]) ** 2) ** 0.5
forest_accuracy = accuracy_score(test[y_col], test["forest_pred"])

print("Training set error for random forest:", train_rmse)
print("Test set error for random forest:    ", test_rmse)
print("Accuracy:", forest_accuracy)

Training set error for random forest: 0.27939053622117876
Test set error for random forest:     0.6193930890332394
Accuracy: 0.6163522012578616


Nonparametric models: Decision Tree

In [141]:
from sklearn.tree import DecisionTreeClassifier


tree_model = DecisionTreeClassifier()

tree_model.fit(train[X_cols], train[y_col])

train["tree_pred"] = tree_model.predict(train[X_cols])
test["tree_pred"] = tree_model.predict(test[X_cols])

In [142]:
train_rmse = np.mean((train["tree_pred"] - train[y_col]) ** 2) ** 0.5
test_rmse = np.mean((test["tree_pred"] - test[y_col]) ** 2) ** 0.5

print("Training set error for decision tree:", train_rmse)
print("Test set error for decision tree:    ", test_rmse)


tree_accuracy = accuracy_score(test[y_col], test["tree_pred"])
print("Accuracy:", tree_accuracy)



Training set error for decision tree: 0.27939053622117876
Test set error for decision tree:     0.6344412685745153
Accuracy: 0.5974842767295597


```
Evaluation Metrics:

Besides RMSE and accuracy, consider other metrics like precision, recall, F1 score, especially for imbalanced datasets.

Model Complexity:

Random Forest's max_features=1 might be too restrictive.
Try adjusting max_features and other hyperparameters like n_estimators and max_depth.
Model Overfitting:

Both models might be overfitting to the training data.
Consider implementing cross-validation to better understand model performance.
```

## Looking into Prevalence

In [143]:
#number of rows in house to beign with that actually won. 
prevalence = len(house[house['General Status']==1])/len(house)
print("Prevalence of wins:", prevalence)

Prevalence of wins: 0.2685624012638231


<!-- END QUESTION -->

## 2b) Frequentist Regression

Let's start by considering the problem from a frequentist lens. To do this, we'll use the `statsmodels.api`, which allows us to create a model in just a few lines of code.

After fitting our model, we can call the `.summary()` method, and get a breakdown of our model and some details on how well it fit our data.

In [ ]:
import statsmodels.api as sm

In [1]:
#FROM LAB 
# Fit Poisson GLM model where Temp_Anomaly is a covariate (exogenous variable): No need to modify
freq_model = sm.GLM(df["Num_Storms"], exog = sm.add_constant(df["Temp_Anomaly"]), 
                  family=sm.families.Poisson())
freq_res = freq_model.fit()
print(freq_res.summary())

NameError: name 'sm' is not defined

In [ ]:
model_two = sm.OLS(np.log(Y), sm.add_constant(X)).fit()
print(model_two.summary())